In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Đọc file Excel gốc
df = pd.read_excel(r"C:\Users\Admin\Documents\python\datasetprj_2012.xlsx")
## ee lấy đường dẫn nnay đúng k v

# Làm sạch cột Order_Demand
df['Order_Demand'] = pd.to_numeric(df['Order_Demand'], errors='coerce')
df.dropna(subset=['Order_Demand'], inplace=True)
df['Order_Demand'] = df['Order_Demand'].astype(int)

# Dữ liệu Order_Demand đã sẵn sàng để vẽ
demand_data = df['Order_Demand']

# Vẽ Biểu đồ Histogram
# Điều này giúp mỗi thanh đại diện cho một mức cầu cố định (ví dụ: 100, 500, 1000)
bins = demand_data.nunique() 

plt.figure(figsize=(10, 6))
plt.hist(demand_data, bins=bins, edgecolor='black', alpha=0.7)
plt.title('Phân phối Order Demand Tổng Thể', fontsize=16, fontweight='bold')
plt.xlabel('Mức Cầu Đặt Hàng Cố Định', fontsize=12)
plt.ylabel('Freq', fontsize=12)
plt.grid(axis='y', alpha=0.5)

print("-" * 50)
print("Đang hiển thị biểu đồ...")
plt.show()

## nxet: Right-Skewed and Discrete Distribution; longtail; ko dung duoc normal
#    Vì nhu cầu là rời rạc, mô hình tối ưu nên tập trung vào việc tính toán tồn kho an toàn để đáp ứng 
# các mức cầu cố định tiếp theo (ví dụ: cần bao nhiêu hàng để đáp ứng mức cầu 500 nếu đã hết hàng ở mức 100).

#    Quản lý Cầu Đột biến: Cần sử dụng các mô hình Tối ưu hóa Mạnh mẽ (Robust Optimization) 
# hoặc các mô hình Quy hoạch Động (Dynamic Programming) để cân bằng chi phí giữa:

#    Chi phí giữ hàng (Holding Cost): Dành cho các mặt hàng nhỏ, thường xuyên.

#    Chi phí thiếu hàng (Stockout Cost): Khi không đáp ứng được các đơn hàng lớn, hiếm gặp ở đuôi phải.

#  Tóm lại: Biểu đồ này chỉ ra một cấu trúc bán hàng cố định (theo lô hàng) và một rủi ro thiếu hàng cao do các đơn hàng lớn,
#  đòi hỏi mô hình tối ưu hóa phải linh hoạt và chính xác hơn các phương pháp thống kê truyền thống.


 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Admin\\Documents\\python\\datasetprj_2012.xlsx'